<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/CI_CD_GitHubAction_Heroku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To set up a GitHub Action that trains a machine learning model on the Iris dataset, saves it as an artifact, and deploys the app on Heroku, you can follow these steps:

###Make sure your GitHub repository contains the following files:
- train_model.py: A Python script to train the model and save it as a .pkl file.- app.py: A Python script containing a Flask or FastAPI app that uses the trained model to make predictions. 


- requirements.txt: A list of Python dependencies.
- Procfile: A Heroku Procfile to specify the app's entry point.
- runtime.txt: A file to specify the Python runtime version.

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*HY_qNXxMeD3LyUaXZHlcug.jpeg)

In [ ]:
#train_model.py
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

def train_model():
    iris = load_iris()
    X, y = iris.data, iris.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    return clf

if __name__ == "__main__":
    model = train_model()
    joblib.dump(model, "model.pkl")

In [ ]:
#app.py
import joblib
from flask import Flask, jsonify, request

app = Flask(__name__)
model = joblib.load("model.pkl")

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    features = data["features"]
    prediction = model.predict([features])
    return jsonify({"prediction": int(prediction[0])})

if __name__ == "__main__":
    app.run()

In [ ]:
#requirements.txt
flask
joblib
scikit-learn
pandas
gunicorn

In [ ]:
#Procfile
web: gunicorn app:app

In [ ]:
#runtime.txt
python-3.8

###Create a new file in your repository named .github/workflows/main.yml. This is the GitHub Actions workflow configuration file.

In [ ]:
#Add the following content to the main.yml file:

name: Train Model and Deploy to Heroku

on:
  push:
    branches:
      - main

jobs:
  train:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout repository
      uses: actions/checkout@v2

    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.8

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
    - name: Train model
      run: python train.py

    - name: Save Model
      uses: actions/upload-artifact@v2
      with:
        name: model
        path: model.pkl

  deploy:
    runs-on: ubuntu-latest
    needs: train

    steps:
    - name: Checkout repository
      uses: actions/checkout@v2

    - name: Download model artifact
      uses: actions/download-artifact@v2
      with:
        name: model
        path: model.pkl

    - name: Install Heroku CLI
      run: curl https://cli-assets.heroku.com/install.sh | sh

    - name: Deploy to Heroku
      run: |
        cd ${{ github.workspace }}
        heroku container:login
        heroku create iris-heroku-app-mlops
        git push heroku HEAD:main
      env:
        HEROKU_API_KEY: ${{ secrets.HEROKU_API_KEY }}
        HEROKU_APP_NAME: "your-heroku-app-name"
        HEROKU_EMAIL: "your@email.com"

Replace your-heroku-app-name with your desired Heroku app name and your@email.com with your Heroku account's email address. Add your Heroku API key as a secret in your GitHub repository settings.

Commit and push these files to the main branch of your repository. The GitHub Action will train the model, save it as an artifact, and deploy the Flask app to Heroku.

You can then make predictions by sending a POST request to your Heroku app at https://your-heroku-app-name.herokuapp.com/predict with the following JSON payload:

In [ ]:
{
    "features": [5.1, 3.5, 1.4, 0.2]
}